In [22]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, types
import datetime
import requests
import time

In [23]:
username = 'TALENT_LMS_KEY'
password = ''


mysql_host = 'HOSTNAME'
mysql_root = 'USERNAME'
mysql_password = 'PASSWORD'
mysql_db = 'DATABASE_NAME'


mysql_engine = create_engine(f'mysql+pymysql://{mysql_root}:{mysql_password}@{mysql_host}:3306/{mysql_db}')

In [26]:
def load_user_enrolled_courses():
    conn = mysql_engine.connect()
    result = conn.execute(f"select id from {mysql_db}.talent_lms_courses") # Want to change the query

    FINAL_DATAFRAMES = []
    for i in result:
        retries = 1
        success = False
    
        while not success:
            try:
                course_id = i[0]
                url = f'https://thirdspace.talentlms.com/api/v1/courses/id:{course_id}'
                response = requests.get(url, auth=(username, password), timeout=5)
                course_data = response.json()
                success = True

            except requests.exceptions.ConnectionError:
                wait = retries * 5
                print(f'Error! Waiting {wait} secs and re-trying...')
                time.sleep(wait)
                retries += 1

        df = pd.json_normalize(course_data['users'])
        df['course_id'] = course_id
        FINAL_DATAFRAMES.append(df)
        print('loaded course with course_id:', course_id)

    users_enrolled_courses = pd.concat(FINAL_DATAFRAMES, ignore_index=True)

    users_enrolled_courses['enrolled_on_timestamp'] = users_enrolled_courses['enrolled_on_timestamp'].astype('int64')
    users_enrolled_courses['user_enrolled_at'] = users_enrolled_courses['enrolled_on_timestamp'].\
        apply(lambda x: datetime.datetime.fromtimestamp(x))

    users_enrolled_courses['completed_on_timestamp'] = users_enrolled_courses['completed_on_timestamp'].\
        apply(lambda x: int(0 if x is None else x))
    users_enrolled_courses['user_completed_at'] = users_enrolled_courses['completed_on_timestamp'].\
        apply(lambda x: datetime.datetime.fromtimestamp(x))

    COLS_TO_DROP = ['enrolled_on', 'enrolled_on_timestamp', 'completed_on', 
                    'completed_on_timestamp', 'expired_on', 'expired_on_timestamp']

    lms_user_enrolled_courses = users_enrolled_courses.drop(columns=COLS_TO_DROP)

    cols_rename = {
        'id' : 'user_id',
        'name' : 'user_name',
        'role' : 'user_role',
    }

    lms_user_enrolled_courses = lms_user_enrolled_courses.rename(columns=cols_rename)
    return lms_user_enrolled_courses


def create_temp_user_enrolled_courses_table(df):
    
    sql_types = {
                'user_id' : types.INTEGER(),
                'user_name': types.VARCHAR(length=255),
                'user_role': types.VARCHAR(64),
                'completion_percentage' : types.INTEGER(),
                'total_time': types.VARCHAR(length=255),
                'total_time_seconds': types.INTEGER(),
                'course_id' : types.INTEGER(),
                'user_enrolled_at': types.TIMESTAMP(),
                'user_completed_at': types.TIMESTAMP()   
            }

    df.to_sql(
            name='temp_talent_lms_user_enrolled_courses',
            con=mysql_engine,
            index=False,
            if_exists='replace'
        )

def upsert_user_enrolled_courses_table():
    conn = mysql_engine.connect()
    
    sql_query = f'''
    insert into {mysql_db}.talent_lms_user_enrolled_courses 
    select *, current_timestamp() as row_modified_at from {mysql_db}.temp_talent_lms_user_enrolled_courses
        on duplicate key update 
            talent_lms_user_enrolled_courses.user_name             = temp_talent_lms_user_enrolled_courses.user_name,
            talent_lms_user_enrolled_courses.user_role             = temp_talent_lms_user_enrolled_courses.user_role,
            talent_lms_user_enrolled_courses.completion_percentage = temp_talent_lms_user_enrolled_courses.completion_percentage,
            talent_lms_user_enrolled_courses.total_time            = temp_talent_lms_user_enrolled_courses.total_time,
            talent_lms_user_enrolled_courses.total_time_seconds    = temp_talent_lms_user_enrolled_courses.total_time_seconds,
            talent_lms_user_enrolled_courses.user_enrolled_at      = temp_talent_lms_user_enrolled_courses.user_enrolled_at,
            talent_lms_user_enrolled_courses.user_completed_at     = temp_talent_lms_user_enrolled_courses.user_completed_at,
            talent_lms_user_enrolled_courses.row_modified_at       = current_timestamp();
    '''

    conn.execute(sql_query)

In [18]:
df = load_user_enrolled_courses()
create_temp_user_enrolled_courses_table(df)
upsert_user_enrolled_courses_table()